# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
city_csv= pd.read_csv("output_data_file")
city_df = pd.DataFrame(city_csv)
city_df.head()

,Unnamed: 0,City,Cloudiness,Date,Country,Humidity,Lat,Lon,Max Temp,Wind MPH
0,0,butaritari,0,1594997303,KI,76,3.07,172.79,82.922,6.30
1,1,beloha,18,1594997430,MG,72,-25.17,45.05,63.698,3.04
2,2,mataura,0,1594997296,NZ,97,-46.19,168.86,29.066,0.84
3,3,westport,90,1594997430,US,83,41.14,-73.36,71.600,5.10
4,4,bluff,37,1594997297,NZ,90,-46.60,168.33,36.644,2.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
gmaps.configure(g_key)
weight= city_df["Humidity"].astype(float)
location = city_df[["Lat", "Lon"]].astype(float)

In [33]:
fig=gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=weight, 
                                 dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
narrow_city_weather = city_df.loc[(city_df["Max Temp"]<80) 
                                  &(city_df["Max Temp"]>70)
                                 &(city_df["Wind MPH"]<10)
                                  &(city_df["Cloudiness"]==0)]
hotel_df= narrow_city_weather.drop(columns= "Unnamed: 0")
hotel_df

,City,Cloudiness,Date,Country,Humidity,Lat,Lon,Max Temp,Wind MPH
18,bredasdorp,0,1594997297,ZA,29,-34.53,20.04,77.000,1.50
41,goderich,0,1594997119,CA,84,43.75,-81.72,75.992,3.76
47,hermanus,0,1594997294,ZA,47,-34.42,19.23,71.006,0.45
88,cape town,0,1594996998,ZA,23,-33.93,18.42,77.000,6.20
171,proletariy,0,1594997442,RU,43,58.43,31.70,72.086,3.68
193,mwinilunga,0,1594997444,ZM,29,-11.74,24.43,70.322,5.03
228,saldanha,0,1594997335,ZA,31,-33.01,17.94,73.400,6.20
291,nikolskoye,0,1594997296,RU,33,59.70,30.79,73.400,6.00
298,avare,0,1594997453,BR,54,-23.10,-48.93,75.164,0.95
308,guerrero negro,0,1594997338,MX,71,27.98,-114.06,71.402,4.92


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df["Hotel Name"] = ""
hotel_df


,City,Cloudiness,Date,Country,Humidity,Lat,Lon,Max Temp,Wind MPH,Hotel Name
18,bredasdorp,0,1594997297,ZA,29,-34.53,20.04,77.000,1.50,
41,goderich,0,1594997119,CA,84,43.75,-81.72,75.992,3.76,
47,hermanus,0,1594997294,ZA,47,-34.42,19.23,71.006,0.45,
88,cape town,0,1594996998,ZA,23,-33.93,18.42,77.000,6.20,
171,proletariy,0,1594997442,RU,43,58.43,31.70,72.086,3.68,
193,mwinilunga,0,1594997444,ZM,29,-11.74,24.43,70.322,5.03,
228,saldanha,0,1594997335,ZA,31,-33.01,17.94,73.400,6.20,
291,nikolskoye,0,1594997296,RU,33,59.70,30.79,73.400,6.00,
298,avare,0,1594997453,BR,54,-23.10,-48.93,75.164,0.95,
308,guerrero negro,0,1594997338,MX,71,27.98,-114.06,71.402,4.92,


In [49]:
target_type="hotel"
radius= 5000
hotel_list=[]
for index, row in hotel_df.iterrows():
    latitude= row["Lat"]
    longitude= row["Lon"]
    coords=f"{latitude},{longitude}"
    params={"location": coords,
           "radius": radius,
           "target":target_type,
           "key": g_key
           }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response= requests.get(base_url, params).json()
    #pprint(response["results"][1]["name"])
    hotel_list.append(response["results"][1]["name"])
hotel_df["Hotel Name"]= hotel_list
hotel_df


,City,Cloudiness,Date,Country,Humidity,Lat,Lon,Max Temp,Wind MPH,Hotel Name
18,bredasdorp,0,1594997297,ZA,29,-34.53,20.04,77.000,1.50,Bredasdorp Country Manor
41,goderich,0,1594997119,CA,84,43.75,-81.72,75.992,3.76,Harmony Inn
47,hermanus,0,1594997294,ZA,47,-34.42,19.23,71.006,0.45,Misty Waves Boutique Hotel
88,cape town,0,1594996998,ZA,23,-33.93,18.42,77.000,6.20,Southern Sun Waterfront Cape Town
171,proletariy,0,1594997442,RU,43,58.43,31.70,72.086,3.68,Tri Kita Mihi-Hotel Bronnitsa
193,mwinilunga,0,1594997444,ZM,29,-11.74,24.43,70.322,5.03,Jericho Girls Hostel
228,saldanha,0,1594997335,ZA,31,-33.01,17.94,73.400,6.20,Blue Bay Lodge
291,nikolskoye,0,1594997296,RU,33,59.70,30.79,73.400,6.00,Nikolskaya College № 1
298,avare,0,1594997453,BR,54,-23.10,-48.93,75.164,0.95,Magazine Luiza
308,guerrero negro,0,1594997338,MX,71,27.98,-114.06,71.402,4.92,Bimbo


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [55]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))